# Convolutional Neural Networks (CNNs)

CNNs are **the** tool for imaging tasks.  Convolutions can capture desirable properties of images, such as translation invariance, and are computed efficiently on GPUs.  The number of weights needed for each convolution is independent of the number of layer features, hence CNNs can tackle problems very high dimensional data as well.

### Outcomes 
In this tutorial, you will

*   load the MNIST dataset from torchvision
*   learn how to construct a convolutional neural network
*   learn how to use a GPU for training

### Suggested Activities


*  How does the network architecture affect the performance?  Play with the different parameters in convolutional layers.
*  The MNIST dataset is notoriously easy to classify.  Explore other datasets in PyTorch and see how you can 

Check out the [PyTorch Documentation](https://pytorch.org/docs/stable/index.html) as you explore!

The network used for this notebook follows the example in  https://github.com/pytorch/examples/tree/main/mnist.








## Step 1: Import Packages

We start by importing the necessary packages to run our code.  We are installing the following packages:

   * deep learning toolbox [Pytorch](https://pytorch.org/)
   * vision toolbox [torchvision](https://pytorch.org/vision/stable/index.html)
   * visualization toolbox [Matplotlib](https://matplotlib.org/)
   * DNN101 repository [https://github.com/elizabethnewman/dnn101](https://github.com/elizabethnewman/dnn101).

In [ ]:
!python -m pip install git+https://github.com/elizabethnewman/dnn101.git


In [ ]:
import dnn101
import torch
import torch.nn as nn
from torchvision import datasets, transforms
import matplotlib as mpl
import matplotlib.pyplot as plt

## Step 2: Setup Device

We will check and use the GPU if available. 

You can use a GPU on Google Colab by following these instructions: 

*   Locate the Runtime menu (top left of your screen)
*   Select Runtime -> Change runtime type. A dialogue box will open.  
*   Coose GPU under the Hardware accelarator dropdown list.  
*   Click save and you're on a GPU!

In [ ]:
# create data
use_cuda = torch.cuda.is_available()
device = torch.device("cpu")
if use_cuda:
    device = torch.device("cuda")

train_kwargs = {'batch_size': 32}
test_kwargs = {'batch_size': 32}
if use_cuda:
    cuda_kwargs = {'num_workers': 1, 'pin_memory': True, 'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

## Step 3: Load the Data



In [ ]:
from copy import deepcopy

# data parameters
n_train = 1000
n_val = 100
n_test = 100
seed = 123       # seed for reproducibility

# set seed for reproducibility
torch.manual_seed(123)


# load data with transforms to normalize the images
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
dataset1 = datasets.MNIST('../raw_data', train=True, download=True, transform=transform)
dataset2 = datasets.MNIST('../raw_data', train=False, transform=transform)

# create data loaders
train_loader = torch.utils.data.DataLoader(dataset1, **train_kwargs)
val_loader = deepcopy(train_loader)
val_loader.train = False
val_loader.shuffle = False
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

# create smaller datasets for faster experimentation if preferred
if n_train is not None:
    train_idx = torch.randperm(train_loader.dataset.data.shape[0])
    train_loader.dataset.data = train_loader.dataset.data[train_idx[:n_train]]
    train_loader.dataset.targets = train_loader.dataset.targets[train_idx[:n_train]]

if n_val is not None:
    val_loader.dataset.data = val_loader.dataset.data[train_idx[n_train:n_train + n_val]]
    val_loader.dataset.targets = val_loader.dataset.targets[train_idx[n_train:n_train + n_val]]

if n_test is not None:
    # test
    test_idx = torch.randperm(test_loader.dataset.data.shape[0])
    test_loader.dataset.data = test_loader.dataset.data[test_idx[:n_test]]
    test_loader.dataset.targets = test_loader.dataset.targets[test_idx[:n_test]]


## Step 3: Train!

This block is the heart of DNN training and consists of four main steps
1. Define the architecture of the DNN (for this notebook, we choose [Linear Layers](https://pytorch.org/docs/stable/nn.html#linear-layers) and [Activation Functions](https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity))
2. Choose a loss function ([PyTorch Loss Functions](https://pytorch.org/docs/stable/nn.html#loss-functions)).  For regression, we use the [Mean Squared Error Loss](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html#torch.nn.MSELoss). 
3. Choose an optimizer ([PyTorch Optimizers](https://pytorch.org/docs/stable/optim.html?highlight=optim#torch.optim.Optimizer))
4. Train with stochastic optimization.  

If you change parameters, be sure to run all blocks in this section to make sure everything is connected properly.

### 1. Architecture

Take a look at the documentation for the new layers!


*   [Conv2d](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html): apply a translation-invariant operator with a small number of weights efficiently on a GPU
*   [MaxPool2d](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html): effectively a coarsening of the image pixels
*   [Dropout](https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html): effectively a way to avoid overfitting



Note that the MNIST images are stored as $(N, 1, 28, 28)$, meaning we have $N$ images of size $28 \times 28$ with $1$ channel each (i.e., the images are grayscale).

In [ ]:
# set seed for reproducibility
torch.manual_seed(42)

# a reshaping layer for convenience
class View(nn.Module):
    def __init__(self, shape):
        super(View, self).__init__()
        self.shape = shape

    def forward(self, x):
        return x.view(*self.shape)


# create architecture
net = nn.Sequential(
    nn.Conv2d(1, 32, 3, 1),
    nn.ReLU(),
    nn.Conv2d(32, 64, 3, 1),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Dropout(0.25),
    View((-1, 12 * 12 * 64)),
    nn.Linear(12 * 12 * 64, 128),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(128, 10)
).to(device)

In [ ]:
from dnn101.cnn.plotting import plot_Conv2d_filters, plot_CNN_features

print(net)
plot_Conv2d_filters(net, 16)
plt.show()


x, _ = next(iter(train_loader))
plot_CNN_features(net, x[0:1].to(device))
plt.show()

plot_CNN_features(net, x[3:4].to(device), n_features=16)
plt.show()

### 2. Loss Function

In PyTorch, the default [cross entropy loss function](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)  given by
\begin{align*}
L(f(\mathbf{y},\mathbf{\theta}), c) = -\mathbf{e}_c^\top \log\sigma(f(\mathbf{y},\mathbf{\theta}))]
\end{align*}
where $c$ is the class to which $\mathbf{y}$ belongs, $\mathbf{e}_c$ is the standard basis vector with a $1$ in the $c$-th component.  The softmax function $\sigma$ is given by 
  \begin{align*}
  \sigma(\mathbf{z}) = \frac{\exp(\mathbf{z})}{\mathbf{e}^\top \exp(\mathbf{z})}
  \end{align*}
where $\mathbf{e}$ is the constant vector of all ones. 

In [ ]:
loss = torch.nn.CrossEntropyLoss() 

### 3. Optimizer and Scheduler

In [ ]:
optimizer = torch.optim.Adam(net.parameters(), lr=1e-2)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, 0.9)

### 4. Train!

In [ ]:
from IPython import display
mpl.rcParams['figure.figsize'] = (16, 6)
mpl.rcParams['lines.linewidth'] = 8
mpl.rcParams['font.size'] = 10

# import training
from dnn101.utils import evaluate, train

# printing and plotting options (only one can be True)
verbose = False             # printouts
show_plot = not verbose     # plot to show training

# set seed for reproducibility
torch.manual_seed(42)

# store and print results
info = {
    'headers': ('epoch', 'lr', 'run_loss', 'run_acc', 'train_loss', 'train_acc', 'valid_loss', 'valid_acc'),
    'formats': '{:<15d}{:<15.4e}{:<15.4e}{:<15.4f}{:<15.4e}{:<15.4f}{:<15.4e}{:<15.4f}',
    'values': []
    }

loss_train, acc_train = evaluate(net, loss, train_loader, device=device)
loss_val, acc_val = evaluate(net, loss, val_loader, device=device)
info['values'].append([-1, optimizer.param_groups[0]['lr'], 0.0, 0.0, loss_train, acc_train, loss_val, acc_val])

if verbose:
    print(('{:<15s}' * len(info['headers'])).format(*info['headers']))
    print(info['formats'].format(*info['values'][-1]))

max_epochs = 10
for epoch in range(max_epochs):
    # learn
    loss_running, acc_running = train(net, loss, train_loader, optimizer, device=device)

    # evaluate
    loss_train, acc_train = evaluate(net, loss, train_loader, device=device)
    loss_val, acc_val = evaluate(net, loss, val_loader, device=device)

    # store and print or plot results
    info['values'].append([epoch, optimizer.param_groups[0]['lr'], loss_running, acc_running, loss_train, acc_train, loss_val, acc_val])
    if verbose:
        print(info['formats'].format(*info['values'][-1]))

    if show_plot:
      # show same sample
      next(x for i, (x, _) in enumerate(val_loader) if i <= 0)
      plot_CNN_features(net, x[5:6].to(device),  16)
      plt.title('epoch = %d' % epoch)

      display.display(plt.gcf())
      display.clear_output(wait=True)
      plt.clf()

      
mpl.rcParams.update(mpl.rcParamsDefault)

## Step 4: Inference


In [ ]:
# final losses and accuracies
loss_train, acc_train = evaluate(net, loss, train_loader, device=device)
loss_val, acc_val = evaluate(net, loss, val_loader, device=device)
loss_test, acc_test = evaluate(net, loss, test_loader, device=device)

print('Train: Loss = %0.4e, Acc = %0.2f' % (loss_train, acc_train))
print('Valid: Loss = %0.4e, Acc = %0.2f' % (loss_val, acc_val))
print('Test: Loss = %0.4e, Acc = %0.2f' % (loss_test, acc_test))

In [ ]:
# plotting the features of some samples
next(x for i, (x, _) in enumerate(val_loader) if i <= 0)
plot_CNN_features(net, x[5:6].to(device),  16)
plt.title('sample 1')
plt.show()

plot_CNN_features(net, x[10:11].to(device),  16)
plt.title('sample 2')
plt.show()

plot_CNN_features(net, x[30:31].to(device),  16)
plt.title('sample 3')
plt.show()

In [ ]:
# plotting some convolutional filters
plot_Conv2d_filters(net, 16)
plt.show()